## First I will scrape the pdf. 

In [1]:
!pip install pytesseract pillow pandas



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import geopandas as gpd
from shapely.geometry import Point

## Ok I have cleaned it slightly on csv and double checked if it matches the original pdf. It does! 

In [8]:
import pandas as pd
df_2=pd.read_csv('superfund.csv')
df_2

,Region,State,Site Name,Site ID,EPA ID,Address,City,Zip,County,FF Ind,NAI,Native American Entity (NAI Status),Latitude,Longitude,NPL Status Date
0,1.0,CT,BARKHAMSTED-NEW HARTFORD LANDFILL,100255.0,CTD980732333,ROUTE 44,BARKHAMSTED,6063.0,LITCHFIELD,N,N,NaN,41.893947,-72.989337,1989-10-04 0:00:00
1,1.0,CT,BEACON HEIGHTS LANDFILL,100180.0,CTD072122062,BLACKBERRY HILL ROAD,BEACON FALLS,6403.0,NEW HAVEN,N,N,NaN,41.431950,-73.035281,1983-09-08 0:00:00
2,1.0,CT,DURHAM MEADOWS,100108.0,CTD001452093,MAIN ST,DURHAM,6422.0,MIDDLESEX,N,N,NaN,41.481110,-72.681388,1989-10-04 0:00:00
3,1.0,CT,GALLUP'S QUARRY,100201.0,CTD108960972,ROUTE 12,PLAINFIELD,6374.0,WINDHAM,N,N,NaN,41.665281,-71.924161,1989-10-04 0:00:00
4,1.0,CT,KELLOGG-DEERING WELL FIELD,100252.0,CTD980670814,NORWALK WATER DEPARTMENT,NORWALK,6856.0,FAIRFIELD,N,N,NaN,41.130550,-73.431950,1984-09-21 0:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,10.0,WA,"WESTERN PROCESSING CO., INC.",1000662.0,WAD009487513,7215 S 196TH ST,KENT,98031.0,KING,N,Y,Muckleshoot Indian Tribe,47.425000,-122.241700,1983-09-08 0:00:00
2168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(Current),NaN,NaN,NaN
2169,10.0,WA,WYCKOFF CO./EAGLE HARBOR,1000612.0,WAD009248295,5350 CREOSOTE PL NE,BAINBRIDGE ISLAND,98110.0,KITSAP,N,Y,Suquamish Indian Tribe of,47.621669,-122.516700,1987-07-22 0:00:00
2170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the Port Madison Reservation,NaN,NaN,NaN


## Which states have the most superfund sites 

In [9]:
df_2['State'].value_counts().head(20)

State
NJ    114
CA     97
PA     91
NY     83
MI     65
TX     55
FL     52
WA     47
IL     46
IN     40
NC     38
OH     37
WI     36
MO     33
MA     32
VA     29
SC     28
MN     25
MD     21
CO     20
Name: count, dtype: int64

## Time to convert df to geo dfs EPSG:5070 – NAD83

In [41]:
import geopandas as gpd
from shapely.geometry import Point

df_2['geometry'] = df_2.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)


In [42]:
gdf = gpd.GeoDataFrame(df_2, geometry='geometry', crs='EPSG:4326')

In [43]:
gdf = gdf.to_crs(epsg=5070)

In [44]:

gdf['buffered'] = gdf.geometry.buffer(1500)


In [45]:
geometry = [Point(xy) for xy in zip(df_2['Longitude'], df_2['Latitude'])]


In [46]:
gdf = gpd.GeoDataFrame(df_2, geometry=geometry, crs="EPSG:4326")


In [47]:
gdf_5070 = gdf.to_crs(epsg=5070)

In [48]:
gdf_5070.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [49]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [50]:
gdf = gdf.to_crs(epsg=5070)



In [51]:
# Reproject to a projected CRS for accurate buffering
gdf = gdf.to_crs(epsg=5070)

# Save the original point geometry in a new column
gdf['original_point'] = gdf.geometry

gdf['buffer_1609.34m'] = gdf['original_point'].buffer(1500)


In [52]:
gdf = gdf.set_geometry('buffer_1609.34')

## Now I want to get out my private school and public school data. 

### Here is where I got my public school dataset from: https://nces.ed.gov/programs/edge/geographic/schoollocations

In [28]:
df_1=pd.read_excel('publicschools.xlsx')
df_1

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,LON,CBSA,NMCBSA,CBSATYPE,CSA,NMCSA,CD,SLDL,SLDU,SCHOOLYEAR
0,10000500870,100005,Albertville Middle School,1,600 E Alabama Ave,Albertville,AL,35950,1,1095,...,-86.206200,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
1,10000500871,100005,Albertville High School,1,402 E McCord Ave,Albertville,AL,35950,1,1095,...,-86.204900,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
2,10000500879,100005,Albertville Intermediate School,1,901 W McKinney Ave,Albertville,AL,35950,1,1095,...,-86.220100,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
3,10000500889,100005,Albertville Elementary School,1,145 West End Drive,Albertville,AL,35950,1,1095,...,-86.221806,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
4,10000501616,100005,Albertville Kindergarten and PreK,1,257 Country Club Rd,Albertville,AL,35951,1,1095,...,-86.193300,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102269,780003000024,7800030,Lockhart Elementary School,78,41 ESTATE THOMAS,Saint Thomas,VI,802,78,78030,...,-64.917602,N,N,0,N,N,7898,N,N,2023-2024
102270,780003000026,7800030,Ulla F. Muller Elementary School,78,7B ESTATE CONTANT,Saint Thomas,VI,802,78,78030,...,-64.952483,N,N,0,N,N,7898,N,N,2023-2024
102271,780003000027,7800030,Yvonne E. Milliner-Bowsky Elementary School,78,15B and 16 ESTATE MANDAHL,Saint Thomas,VI,802,78,78030,...,-64.899024,N,N,0,N,N,7898,N,N,2023-2024
102272,780003000033,7800030,Addelita Cancryn School,78,1 CROWN BAY,Saint Thomas,VI,802,78,78030,...,-64.945940,N,N,0,N,N,7898,N,N,2023-2024


## Yay! Now time to convert this public school datasetto geo dfs EPSG:5070 – NAD83. 

In [31]:
geometry = [Point(xy) for xy in zip(df_1['LON'], df_1['LAT'])]
gdf_school = gpd.GeoDataFrame(df_1, geometry=geometry, crs="EPSG:4326")
gdf_school = gdf_school.to_crs(epsg=5070)
gdf_5070_school = gdf_school.to_crs(epsg=5070)
gdf_5070_school.crs

In [32]:
gdf_school = gpd.GeoDataFrame(df_1, geometry=geometry, crs="EPSG:4326")


In [33]:
gdf_school = gdf_school.to_crs(epsg=5070)

In [34]:
gdf_5070_school = gdf_school.to_crs(epsg=5070)

In [35]:
gdf_5070_school.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [53]:
joined = gpd.sjoin(gdf_school, gdf, how='left', predicate='intersects')
joined

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,Zip,County,FF Ind,NAI,Native American Entity (NAI Status),Latitude,Longitude,NPL Status Date,geometry_right,original_point
0,10000500870,100005,Albertville Middle School,1,600 E Alabama Ave,Albertville,AL,35950,1,1095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,10000500871,100005,Albertville High School,1,402 E McCord Ave,Albertville,AL,35950,1,1095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,10000500879,100005,Albertville Intermediate School,1,901 W McKinney Ave,Albertville,AL,35950,1,1095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,10000500889,100005,Albertville Elementary School,1,145 West End Drive,Albertville,AL,35950,1,1095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,10000501616,100005,Albertville Kindergarten and PreK,1,257 Country Club Rd,Albertville,AL,35951,1,1095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102269,780003000024,7800030,Lockhart Elementary School,78,41 ESTATE THOMAS,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
102270,780003000026,7800030,Ulla F. Muller Elementary School,78,7B ESTATE CONTANT,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
102271,780003000027,7800030,Yvonne E. Milliner-Bowsky Elementary School,78,15B and 16 ESTATE MANDAHL,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
102272,780003000033,7800030,Addelita Cancryn School,78,1 CROWN BAY,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [55]:
joined['distance_to_site_m'] = joined.geometry.distance(joined['original_point'])
joined['distance_to_site_m']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
102269   NaN
102270   NaN
102271   NaN
102272   NaN
102273   NaN
Name: distance_to_site_m, Length: 102402, dtype: float64

In [56]:
closest = joined.sort_values('distance_to_site_m')
closest

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,County,FF Ind,NAI,Native American Entity (NAI Status),Latitude,Longitude,NPL Status Date,geometry_right,original_point,distance_to_site_m
58961,350006000078,3500060,LONGFELLOW ELEMENTARY,35,400 EDITH NE,ALBUQUERQUE,NM,87102,35,35001,...,BERNALILLO,N,N,NaN,35.086444,-106.640611,1999-10-22 0:00:00,POINT (-960066.11 1390277.447),POINT (-960066.11 1390277.447),73.607322
96930,530702003665,5307020,Insight School of WA Open Doors Program,53,2601 S 35th St Ste 100,Tacoma,WA,98409,53,53053,...,PIERCE,N,Y,Puyallup Tribe of the,47.229439,-122.471400,1983-09-08 0:00:00,POINT (-1989580.15 2971974.078),POINT (-1989580.15 2971974.078),81.769240
69446,390449201858,3904492,Van Cleve Elementary School,39,617 E Main St,Troy,OH,45373,39,39109,...,MIAMI,N,N,NaN,40.038100,-84.196100,2008-09-03 0:00:00,POINT (995781.563 1953168.432),POINT (995781.563 1953168.432),106.703308
65689,370162000701,3701620,North Belmont Elementary,37,210 School Street,Belmont,NC,28012,37,37071,...,GASTON,N,N,NaN,35.273472,-81.051250,1999-07-22 0:00:00,POINT (1342751.256 1463228.861),POINT (1342751.256 1463228.861),109.729611
102213,720003002019,7200030,INES MARIA MENDOZA DE MUNOZ MARIN CABO ROJO,72,CALLE BARBOSA NORTE URB ANA MARIA,CABO ROJO,PR,623,72,72023,...,CABO ROJO,N,N,NaN,18.090725,-67.148325,2011-03-10 0:00:00,POINT (3126218.406 -50039.378),POINT (3126218.406 -50039.378),112.137600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102269,780003000024,7800030,Lockhart Elementary School,78,41 ESTATE THOMAS,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
102270,780003000026,7800030,Ulla F. Muller Elementary School,78,7B ESTATE CONTANT,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
102271,780003000027,7800030,Yvonne E. Milliner-Bowsky Elementary School,78,15B and 16 ESTATE MANDAHL,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
102272,780003000033,7800030,Addelita Cancryn School,78,1 CROWN BAY,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN


In [58]:
df = closest.dropna(subset=['distance_to_site_m'])
df

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,County,FF Ind,NAI,Native American Entity (NAI Status),Latitude,Longitude,NPL Status Date,geometry_right,original_point,distance_to_site_m
58961,350006000078,3500060,LONGFELLOW ELEMENTARY,35,400 EDITH NE,ALBUQUERQUE,NM,87102,35,35001,...,BERNALILLO,N,N,NaN,35.086444,-106.640611,1999-10-22 0:00:00,POINT (-960066.11 1390277.447),POINT (-960066.11 1390277.447),73.607322
96930,530702003665,5307020,Insight School of WA Open Doors Program,53,2601 S 35th St Ste 100,Tacoma,WA,98409,53,53053,...,PIERCE,N,Y,Puyallup Tribe of the,47.229439,-122.471400,1983-09-08 0:00:00,POINT (-1989580.15 2971974.078),POINT (-1989580.15 2971974.078),81.769240
69446,390449201858,3904492,Van Cleve Elementary School,39,617 E Main St,Troy,OH,45373,39,39109,...,MIAMI,N,N,NaN,40.038100,-84.196100,2008-09-03 0:00:00,POINT (995781.563 1953168.432),POINT (995781.563 1953168.432),106.703308
65689,370162000701,3701620,North Belmont Elementary,37,210 School Street,Belmont,NC,28012,37,37071,...,GASTON,N,N,NaN,35.273472,-81.051250,1999-07-22 0:00:00,POINT (1342751.256 1463228.861),POINT (1342751.256 1463228.861),109.729611
102213,720003002019,7200030,INES MARIA MENDOZA DE MUNOZ MARIN CABO ROJO,72,CALLE BARBOSA NORTE URB ANA MARIA,CABO ROJO,PR,623,72,72023,...,CABO ROJO,N,N,NaN,18.090725,-67.148325,2011-03-10 0:00:00,POINT (3126218.406 -50039.378),POINT (3126218.406 -50039.378),112.137600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58771,341806006062,3418060,Winslow Township Elementary School Five,34,130 OAK LEAF ROAD,BERLIN,NJ,8009,34,34007,...,CAMDEN,N,N,NaN,39.729722,-74.905917,1999-10-22 0:00:00,POINT (1777082.52 2054730.903),POINT (1777082.52 2054730.903),1496.034427
39281,220189001436,2201890,Doyline High School,22,376 College St,Doyline,LA,71023,22,22119,...,WEBSTER,Y,N,NaN,32.542200,-93.421000,1989-03-13 0:00:00,POINT (240865.932 1055489.72),POINT (240865.932 1055489.72),1496.412352
56714,340312001028,3403120,Eleanor Rush Intermediate School,34,1200 WYNWOOD DRIVE,CINNAMINSON,NJ,8077,34,34005,...,BURLINGTON,N,N,NaN,40.017500,-74.986200,1986-06-10 0:00:00,POINT (1763357.294 2084693.858),POINT (1763357.294 2084693.858),1496.562527
61041,360011901913,3600119,A-TECH HIGH SCHOOL,36,50 BEDFORD AVE,BROOKLYN,NY,11222,36,36047,...,KINGS,N,N,NaN,40.725409,-73.935904,2022-03-16 0:00:00,POINT (1831694.743 2181835.567),POINT (1831694.743 2181835.567),1498.276832


In [61]:
df.to_csv('schoolstonpl_1.csv')

## Analysis of this csv now! 

In [87]:
import pandas as pd
df=pd.read_csv('schoolstonpl_1.csv')
df

,Unnamed: 0,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,...,County,FF Ind,NAI,Native American Entity (NAI Status),Latitude,Longitude,NPL Status Date,geometry_right,original_point,distance_to_site_m
0,58961,350006000078,3500060,LONGFELLOW ELEMENTARY,35,400 EDITH NE,ALBUQUERQUE,NM,87102,35,...,BERNALILLO,N,N,NaN,35.086444,-106.640611,1999-10-22 0:00:00,POINT (-960066.1096148212 1390277.4468398017),POINT (-960066.1096148212 1390277.4468398017),73.607322
1,96930,530702003665,5307020,Insight School of WA Open Doors Program,53,2601 S 35th St Ste 100,Tacoma,WA,98409,53,...,PIERCE,N,Y,Puyallup Tribe of the,47.229439,-122.471400,1983-09-08 0:00:00,POINT (-1989580.1495779958 2971974.077544025),POINT (-1989580.1495779958 2971974.077544025),81.769240
2,69446,390449201858,3904492,Van Cleve Elementary School,39,617 E Main St,Troy,OH,45373,39,...,MIAMI,N,N,NaN,40.038100,-84.196100,2008-09-03 0:00:00,POINT (995781.5631122614 1953168.431587028),POINT (995781.5631122614 1953168.431587028),106.703308
3,65689,370162000701,3701620,North Belmont Elementary,37,210 School Street,Belmont,NC,28012,37,...,GASTON,N,N,NaN,35.273472,-81.051250,1999-07-22 0:00:00,POINT (1342751.2562493836 1463228.8610467175),POINT (1342751.2562493836 1463228.8610467175),109.729611
4,102213,720003002019,7200030,INES MARIA MENDOZA DE MUNOZ MARIN CABO ROJO,72,CALLE BARBOSA NORTE URB ANA MARIA,CABO ROJO,PR,623,72,...,CABO ROJO,N,N,NaN,18.090725,-67.148325,2011-03-10 0:00:00,POINT (3126218.4055054914 -50039.3777737282),POINT (3126218.4055054914 -50039.3777737282),112.137600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1787,58771,341806006062,3418060,Winslow Township Elementary School Five,34,130 OAK LEAF ROAD,BERLIN,NJ,8009,34,...,CAMDEN,N,N,NaN,39.729722,-74.905917,1999-10-22 0:00:00,POINT (1777082.5203648056 2054730.9026897044),POINT (1777082.5203648056 2054730.9026897044),1496.034427
1788,39281,220189001436,2201890,Doyline High School,22,376 College St,Doyline,LA,71023,22,...,WEBSTER,Y,N,NaN,32.542200,-93.421000,1989-03-13 0:00:00,POINT (240865.93200785632 1055489.7201871455),POINT (240865.93200785632 1055489.7201871455),1496.412352
1789,56714,340312001028,3403120,Eleanor Rush Intermediate School,34,1200 WYNWOOD DRIVE,CINNAMINSON,NJ,8077,34,...,BURLINGTON,N,N,NaN,40.017500,-74.986200,1986-06-10 0:00:00,POINT (1763357.294090909 2084693.8583627252),POINT (1763357.294090909 2084693.8583627252),1496.562527
1790,61041,360011901913,3600119,A-TECH HIGH SCHOOL,36,50 BEDFORD AVE,BROOKLYN,NY,11222,36,...,KINGS,N,N,NaN,40.725409,-73.935904,2022-03-16 0:00:00,POINT (1831694.7433296186 2181835.566633673),POINT (1831694.7433296186 2181835.566633673),1498.276832


In [88]:
df = df.drop('Unnamed: 0', axis=1)
df

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,County,FF Ind,NAI,Native American Entity (NAI Status),Latitude,Longitude,NPL Status Date,geometry_right,original_point,distance_to_site_m
0,350006000078,3500060,LONGFELLOW ELEMENTARY,35,400 EDITH NE,ALBUQUERQUE,NM,87102,35,35001,...,BERNALILLO,N,N,NaN,35.086444,-106.640611,1999-10-22 0:00:00,POINT (-960066.1096148212 1390277.4468398017),POINT (-960066.1096148212 1390277.4468398017),73.607322
1,530702003665,5307020,Insight School of WA Open Doors Program,53,2601 S 35th St Ste 100,Tacoma,WA,98409,53,53053,...,PIERCE,N,Y,Puyallup Tribe of the,47.229439,-122.471400,1983-09-08 0:00:00,POINT (-1989580.1495779958 2971974.077544025),POINT (-1989580.1495779958 2971974.077544025),81.769240
2,390449201858,3904492,Van Cleve Elementary School,39,617 E Main St,Troy,OH,45373,39,39109,...,MIAMI,N,N,NaN,40.038100,-84.196100,2008-09-03 0:00:00,POINT (995781.5631122614 1953168.431587028),POINT (995781.5631122614 1953168.431587028),106.703308
3,370162000701,3701620,North Belmont Elementary,37,210 School Street,Belmont,NC,28012,37,37071,...,GASTON,N,N,NaN,35.273472,-81.051250,1999-07-22 0:00:00,POINT (1342751.2562493836 1463228.8610467175),POINT (1342751.2562493836 1463228.8610467175),109.729611
4,720003002019,7200030,INES MARIA MENDOZA DE MUNOZ MARIN CABO ROJO,72,CALLE BARBOSA NORTE URB ANA MARIA,CABO ROJO,PR,623,72,72023,...,CABO ROJO,N,N,NaN,18.090725,-67.148325,2011-03-10 0:00:00,POINT (3126218.4055054914 -50039.3777737282),POINT (3126218.4055054914 -50039.3777737282),112.137600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1787,341806006062,3418060,Winslow Township Elementary School Five,34,130 OAK LEAF ROAD,BERLIN,NJ,8009,34,34007,...,CAMDEN,N,N,NaN,39.729722,-74.905917,1999-10-22 0:00:00,POINT (1777082.5203648056 2054730.9026897044),POINT (1777082.5203648056 2054730.9026897044),1496.034427
1788,220189001436,2201890,Doyline High School,22,376 College St,Doyline,LA,71023,22,22119,...,WEBSTER,Y,N,NaN,32.542200,-93.421000,1989-03-13 0:00:00,POINT (240865.93200785632 1055489.7201871455),POINT (240865.93200785632 1055489.7201871455),1496.412352
1789,340312001028,3403120,Eleanor Rush Intermediate School,34,1200 WYNWOOD DRIVE,CINNAMINSON,NJ,8077,34,34005,...,BURLINGTON,N,N,NaN,40.017500,-74.986200,1986-06-10 0:00:00,POINT (1763357.294090909 2084693.8583627252),POINT (1763357.294090909 2084693.8583627252),1496.562527
1790,360011901913,3600119,A-TECH HIGH SCHOOL,36,50 BEDFORD AVE,BROOKLYN,NY,11222,36,36047,...,KINGS,N,N,NaN,40.725409,-73.935904,2022-03-16 0:00:00,POINT (1831694.7433296186 2181835.566633673),POINT (1831694.7433296186 2181835.566633673),1498.276832


In [89]:
df['STATE'].value_counts()

STATE
NJ    220
CA    192
NY    164
FL    115
MN     96
TX     92
MI     84
PA     67
IN     53
WA     46
IL     46
MA     43
OH     38
WI     36
PR     35
NE     29
NM     29
MO     28
IA     28
TN     21
UT     20
LA     20
VA     20
NC     20
MD     18
DE     18
CO     18
KS     18
NH     17
MT     16
DC     16
CT     16
SC     15
AZ     14
MS     14
WV     14
GA      9
OK      9
VT      8
RI      6
OR      6
AK      4
AR      3
HI      3
NV      2
GU      2
ME      2
AL      1
VI      1
Name: count, dtype: int64

## Cool! Now I will do the same for proposed facilities 

In [70]:
import pandas as pd
df=pd.read_csv('porposed.csv')
df

,Region,State,Site Name,Unnamed: 3,Site ID,EPA ID,Address,Unnamed: 7,City,Unnamed: 9,Zip,County,SAA,FF\nInd,NAI,Native American Entity (NAI Status),Latitude,Longitude,NPL Status Date
0,1.0,MA,GE - HOUSATONIC RIVER,NaN,100460.0,MAD002084093,"BETWEEN PITTSFIELD AND LENOX, MA",NaN,PITTSFIELD,NaN,1201.0,BERKSHIRE,N,N,Y,Mashantucket Pequot,42.450278,-73.232222,09/25/97
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Indian Tribe (Former);,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mashpee Wampanoag,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tribe (Former); Mohegan,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tribe of Indians of,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nation (Current);,NaN,NaN,NaN
81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Confederated Tribes of,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Siletz Indians of Oregon,NaN,NaN,NaN
83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(Current),NaN,NaN,NaN


In [64]:
df = df.dropna(subset=['Site Name'])
df

,Region,State,Site Name,Unnamed: 3,Site ID,EPA ID,Address,Unnamed: 7,City,Unnamed: 9,Zip,County,SAA,FF\nInd,NAI,Native American Entity (NAI Status),Latitude,Longitude,NPL Status Date
0,1.0,MA,GE - HOUSATONIC RIVER,NaN,100460.0,MAD002084093,"BETWEEN PITTSFIELD AND LENOX, MA",NaN,PITTSFIELD,NaN,1201.0,BERKSHIRE,N,N,Y,Mashantucket Pequot,42.450278,-73.232222,09/25/97
12,1.0,NH,MOHAWK TANNERY,NaN,101188.0,NHD981889629,FAIRMONT STREET,NaN,NASHUA,NaN,3060.0,HILLSBOROUGH,N,N,N,NaN,42.765569,-71.487778,05/11/00
13,2.0,NJ,HISTORIC POTTERIES,NaN,203535.0,NJN000203535,Mulberry Street & Breunig Avenue,NaN,TRENTON,NaN,8638.0,MERCER,N,N,N,NaN,40.236541,-74.739361,09/05/24
14,2.0,NJ,ROUTE 561 DUMP,NaN,203909.0,NJ0000453514,ROUTE 561,NaN,GIBBSBORO,NaN,8026.0,CAMDEN,N,N,N,NaN,39.836110,-74.958333,07/28/98
15,3.0,DE,MILLSBORO TCE,NaN,306645.0,DEN000306645,225 WEST DUPONT HIGHWAY,NaN,MILLSBORO,NaN,19966.0,SUSSEX,N,N,N,NaN,38.588158,-75.298061,09/23/09
16,3.0,MD,68TH STREET DUMP/INDUSTRIAL ENTERPRISES,NaN,300338.0,MDD980918387,68TH STREET,NaN,ROSEDALE,NaN,21237.0,BALTIMORE,Y,N,N,NaN,39.307967,-76.517886,04/30/03
17,3.0,PA,FOSTER WHEELER ENERGY CORP./CHURCH,NaN,300671.0,PAD003031788,348 CRESTWOOD ROAD,NaN,Mountain Top,NaN,18707.0,LUZERNE,Y,N,N,NaN,41.133056,-75.878056,04/09/09
18,NaN,NaN,ROAD TCE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,4.0,AL,35TH AVENUE,NaN,410750.0,ALN000410750,3500 35TH N.,NaN,BIRMINGHAM,NaN,35207.0,JEFFERSON,N,N,N,NaN,33.557464,-86.799671,09/22/14
20,4.0,FL,NORMANDY PARK APARTMENTS,NaN,405823.0,FLD984229773,11110 NORTH 56TH STREET,NaN,TEMPLE TERRACE,NaN,33617.0,HILLSBOROUGH,N,N,N,NaN,28.049444,-82.394722,02/13/95


In [71]:
import geopandas as gpd
from shapely.geometry import Point

df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)


In [74]:
gdf_3 = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

In [76]:
gdf_3 = gdf_3.to_crs(epsg=5070)
gdf_3['buffered'] = gdf_3.geometry.buffer(1609.34)
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
gdf_3 = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
gdf_5070 = gdf_3.to_crs(epsg=5070)
gdf_5070.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [77]:
gdf_3.crs
gdf_3 = gdf_3.to_crs(epsg=5070)
gdf_3['buffer_1609.34'] = gdf_3.geometry.buffer(1000)

gdf_3 = gdf_3.to_crs(epsg=5070)


gdf_3['original_point'] = gdf_3.geometry

gdf_3['buffer_1500m'] = gdf_3['original_point'].buffer(1609.34)
gdf_3 = gdf_3.set_geometry('buffer_1609.34')

In [78]:
df_1=pd.read_excel('publicschools.xlsx')
df_1

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,LON,CBSA,NMCBSA,CBSATYPE,CSA,NMCSA,CD,SLDL,SLDU,SCHOOLYEAR
0,10000500870,100005,Albertville Middle School,1,600 E Alabama Ave,Albertville,AL,35950,1,1095,...,-86.206200,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
1,10000500871,100005,Albertville High School,1,402 E McCord Ave,Albertville,AL,35950,1,1095,...,-86.204900,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
2,10000500879,100005,Albertville Intermediate School,1,901 W McKinney Ave,Albertville,AL,35950,1,1095,...,-86.220100,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
3,10000500889,100005,Albertville Elementary School,1,145 West End Drive,Albertville,AL,35950,1,1095,...,-86.221806,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
4,10000501616,100005,Albertville Kindergarten and PreK,1,257 Country Club Rd,Albertville,AL,35951,1,1095,...,-86.193300,10700,"Albertville, AL",2,290,"Huntsville-Decatur-Albertville, AL-TN",104,01026,01009,2023-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102269,780003000024,7800030,Lockhart Elementary School,78,41 ESTATE THOMAS,Saint Thomas,VI,802,78,78030,...,-64.917602,N,N,0,N,N,7898,N,N,2023-2024
102270,780003000026,7800030,Ulla F. Muller Elementary School,78,7B ESTATE CONTANT,Saint Thomas,VI,802,78,78030,...,-64.952483,N,N,0,N,N,7898,N,N,2023-2024
102271,780003000027,7800030,Yvonne E. Milliner-Bowsky Elementary School,78,15B and 16 ESTATE MANDAHL,Saint Thomas,VI,802,78,78030,...,-64.899024,N,N,0,N,N,7898,N,N,2023-2024
102272,780003000033,7800030,Addelita Cancryn School,78,1 CROWN BAY,Saint Thomas,VI,802,78,78030,...,-64.945940,N,N,0,N,N,7898,N,N,2023-2024


In [79]:
geometry = [Point(xy) for xy in zip(df_1['LON'], df_1['LAT'])]
gdf_school = gpd.GeoDataFrame(df_1, geometry=geometry, crs="EPSG:4326")
gdf_school = gdf_school.to_crs(epsg=5070)
gdf_5070_school = gdf_school.to_crs(epsg=5070)
gdf_5070_school.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [80]:
joined = gpd.sjoin(gdf_school, gdf_3, how='left', predicate='intersects')
joined

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,County,SAA,FF\nInd,NAI,Native American Entity (NAI Status),Latitude,Longitude,NPL Status Date,geometry_right,original_point
0,10000500870,100005,Albertville Middle School,1,600 E Alabama Ave,Albertville,AL,35950,1,1095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,10000500871,100005,Albertville High School,1,402 E McCord Ave,Albertville,AL,35950,1,1095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,10000500879,100005,Albertville Intermediate School,1,901 W McKinney Ave,Albertville,AL,35950,1,1095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,10000500889,100005,Albertville Elementary School,1,145 West End Drive,Albertville,AL,35950,1,1095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,10000501616,100005,Albertville Kindergarten and PreK,1,257 Country Club Rd,Albertville,AL,35951,1,1095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102269,780003000024,7800030,Lockhart Elementary School,78,41 ESTATE THOMAS,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
102270,780003000026,7800030,Ulla F. Muller Elementary School,78,7B ESTATE CONTANT,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
102271,780003000027,7800030,Yvonne E. Milliner-Bowsky Elementary School,78,15B and 16 ESTATE MANDAHL,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
102272,780003000033,7800030,Addelita Cancryn School,78,1 CROWN BAY,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [81]:
joined['distance_to_site_m'] = joined.geometry.distance(joined['original_point'])
joined['distance_to_site_m']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
102269   NaN
102270   NaN
102271   NaN
102272   NaN
102273   NaN
Name: distance_to_site_m, Length: 102274, dtype: float64

In [82]:
closest = joined.sort_values('distance_to_site_m')
closest

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,SAA,FF\nInd,NAI,Native American Entity (NAI Status),Latitude,Longitude,NPL Status Date,geometry_right,original_point,distance_to_site_m
58488,341629003248,3416290,Darlene C. McKnight Elementary,34,175 GIRARD AVENUE,TRENTON,NJ,8638,34,34021,...,N,N,N,NaN,40.236541,-74.739361,09/05/24,POINT (1778281.347 2113230.148),POINT (1778281.347 2113230.148),415.487108
92369,490060000988,4900600,Murray Adult Transition,49,147 E 5065 S,MURRAY,UT,84107,49,49035,...,N,N,N,NaN,40.659460,-111.892980,01/18/94,POINT (-1326348.039 2072019.752),POINT (-1326348.039 2072019.752),563.570432
252,10039000139,100390,Hudson KEight School,1,3300 F L Shuttlesworth Dr,Birmingham,AL,35207,1,1073,...,N,N,N,NaN,33.557464,-86.799671,09/22/14,POINT (847075.507 1206569.49),POINT (847075.507 1206569.49),566.486354
82904,480147614076,4801476,PRELUDE PREPARATORY CHARTER SCHOOL,48,1707 CENTENNIAL BLVD,SAN ANTONIO,TX,78211,48,48029,...,N,N,N,NaN,29.372203,-98.537608,06/14/01,POINT (-246410.856 703047.393),POINT (-246410.856 703047.393),569.062479
21632,120087007862,1200870,WOODMONT CHARTER SCHOOL,12,10402 N 56TH ST,TEMPLE TERRACE,FL,33617,12,12057,...,N,N,N,NaN,28.049444,-82.394722,02/13/95,POINT (1337650.129 649337.197),POINT (1337650.129 649337.197),605.888254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102269,780003000024,7800030,Lockhart Elementary School,78,41 ESTATE THOMAS,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
102270,780003000026,7800030,Ulla F. Muller Elementary School,78,7B ESTATE CONTANT,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
102271,780003000027,7800030,Yvonne E. Milliner-Bowsky Elementary School,78,15B and 16 ESTATE MANDAHL,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
102272,780003000033,7800030,Addelita Cancryn School,78,1 CROWN BAY,Saint Thomas,VI,802,78,78030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN


In [83]:
df = closest.dropna(subset=['distance_to_site_m'])
df

,NCESSCH,LEAID,NAME,OPSTFIPS,STREET,CITY,STATE,ZIP,STFIP,CNTY,...,SAA,FF\nInd,NAI,Native American Entity (NAI Status),Latitude,Longitude,NPL Status Date,geometry_right,original_point,distance_to_site_m
58488,341629003248,3416290,Darlene C. McKnight Elementary,34,175 GIRARD AVENUE,TRENTON,NJ,8638,34,34021,...,N,N,N,NaN,40.236541,-74.739361,09/05/24,POINT (1778281.347 2113230.148),POINT (1778281.347 2113230.148),415.487108
92369,490060000988,4900600,Murray Adult Transition,49,147 E 5065 S,MURRAY,UT,84107,49,49035,...,N,N,N,NaN,40.659460,-111.892980,01/18/94,POINT (-1326348.039 2072019.752),POINT (-1326348.039 2072019.752),563.570432
252,10039000139,100390,Hudson KEight School,1,3300 F L Shuttlesworth Dr,Birmingham,AL,35207,1,1073,...,N,N,N,NaN,33.557464,-86.799671,09/22/14,POINT (847075.507 1206569.49),POINT (847075.507 1206569.49),566.486354
82904,480147614076,4801476,PRELUDE PREPARATORY CHARTER SCHOOL,48,1707 CENTENNIAL BLVD,SAN ANTONIO,TX,78211,48,48029,...,N,N,N,NaN,29.372203,-98.537608,06/14/01,POINT (-246410.856 703047.393),POINT (-246410.856 703047.393),569.062479
21632,120087007862,1200870,WOODMONT CHARTER SCHOOL,12,10402 N 56TH ST,TEMPLE TERRACE,FL,33617,12,12057,...,N,N,N,NaN,28.049444,-82.394722,02/13/95,POINT (1337650.129 649337.197),POINT (1337650.129 649337.197),605.888254
27271,160306000485,1603060,HEYBURN ELEMENTARY SCHOOL,16,1405 MAIN AVENUE,ST MARIES,ID,83861,16,16009,...,N,N,Y,Coeur D'Alene Tribe,47.321389,-116.575000,12/01/00,POINT (-1552194.865 2871923.821),POINT (-1552194.865 2871923.821),625.486529
81937,480001608434,4800016,TEXANS CAN ACADEMY - SAN ANTONIO,48,1807 CENTENNIAL BLVD,SAN ANTONIO,TX,78211,48,48029,...,N,N,N,NaN,29.372203,-98.537608,06/14/01,POINT (-246410.856 703047.393),POINT (-246410.856 703047.393),634.741182
92364,490060000374,4900600,Murray High,49,5440 S STATE,MURRAY,UT,84107,49,49035,...,N,N,N,NaN,40.659460,-111.892980,01/18/94,POINT (-1326348.039 2072019.752),POINT (-1326348.039 2072019.752),642.334456
86686,482247002255,4822470,COLUMBIA HEIGHTS EL,48,1610 FITCH AVE,SAN ANTONIO,TX,78211,48,48029,...,N,N,N,NaN,29.372203,-98.537608,06/14/01,POINT (-246410.856 703047.393),POINT (-246410.856 703047.393),654.828794
68775,390438900857,3904389,Dover Middle School,39,2131 N Wooster Ave,Dover,OH,44622,39,39157,...,Y,N,N,NaN,40.539823,-81.494444,05/10/93,POINT (1213542.196 2040250.032),POINT (1213542.196 2040250.032),690.371789


In [84]:
df.to_csv('schools_and_proposednpl.csv')

## Interesting! Now I will figure out how mnay superfund sites are near private school. 

In [107]:
df_4=pd.read_csv('privateschools.csv')
df_4

,X,Y,OBJECTID,PPIN,NAME,STREET,CITY,STATE,ZIP,STFIP,...,NMCBSA,CBSATYPE,CSA,NMCSA,NECTA,NMNECTA,CD,SLDL,SLDU,SCHOOLYEAR
0,-86.541877,32.472090,1,2722,AUTAUGA ACADEMY,497 GOLSON RD,PRATTVILLE,AL,36067,1,...,"Montgomery, AL",1,388,"Montgomery-Selma-Alexander City, AL",N,N,102,1069,1030,2021-2022
1,-86.458475,32.469350,2,A0100060,CAMELLIA BAPTIST WEM,201 WOODVALE RD,PRATTVILLE,AL,36067,1,...,"Montgomery, AL",1,388,"Montgomery-Selma-Alexander City, AL",N,N,102,1088,1030,2021-2022
2,-86.472061,32.460800,3,A0900032,FIRST PRESBYTERIAN KINDERGARTEN,211 S CHESTNUT ST,PRATTVILLE,AL,36067,1,...,"Montgomery, AL",1,388,"Montgomery-Selma-Alexander City, AL",N,N,102,1088,1030,2021-2022
3,-86.466175,32.455425,4,A1592005,NEW HOPE ACADEMY,301 WATER ST,PRATTVILLE,AL,36067,1,...,"Montgomery, AL",1,388,"Montgomery-Selma-Alexander City, AL",N,N,102,1088,1030,2021-2022
4,-86.470441,32.461710,5,K9300030,FIRST BAPTIST PRESCHOOL AND KINDERGARTEN,138 S WASHINGTON ST,PRATTVILLE,AL,36067,1,...,"Montgomery, AL",1,388,"Montgomery-Selma-Alexander City, AL",N,N,102,1088,1030,2021-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22340,-110.810241,43.491060,22341,A0303404,MOUNTAIN ACADEMY OF TETON SCIENCE SCHOOLS,700 COYOTE CANYON RD,JACKSON,WY,83001,56,...,"Jackson, WY-ID",2,N,N,N,N,5600,56023,56017,2021-2022
22341,-110.799526,43.458650,22342,A0903887,JACKSON HOLE COMMUNITY SCHOOL,"1715 HIGH SCHOOL ROAD, #240",JACKSON,WY,83001,56,...,"Jackson, WY-ID",2,N,N,N,N,5600,56016,56017,2021-2022
22342,-110.844286,43.367002,22343,A1102418,RED TOP MEADOWS,7905 FALL CREEK RD,WILSON,WY,83014,56,...,"Jackson, WY-ID",2,N,N,N,N,5600,56022,56016,2021-2022
22343,-110.810901,43.441910,22344,A2000010,JACKSON HOLE CLASSICAL ACADEMY,2500 S PARK LOOP RD,JACKSON,WY,83001,56,...,"Jackson, WY-ID",2,N,N,N,N,5600,56023,56017,2021-2022


In [108]:
df_4.rename(columns={'LAT': 'Latitude', 'LON': 'Longitude'}, inplace=True)
df_4

,X,Y,OBJECTID,PPIN,NAME,STREET,CITY,STATE,ZIP,STFIP,...,NMCBSA,CBSATYPE,CSA,NMCSA,NECTA,NMNECTA,CD,SLDL,SLDU,SCHOOLYEAR
0,-86.541877,32.472090,1,2722,AUTAUGA ACADEMY,497 GOLSON RD,PRATTVILLE,AL,36067,1,...,"Montgomery, AL",1,388,"Montgomery-Selma-Alexander City, AL",N,N,102,1069,1030,2021-2022
1,-86.458475,32.469350,2,A0100060,CAMELLIA BAPTIST WEM,201 WOODVALE RD,PRATTVILLE,AL,36067,1,...,"Montgomery, AL",1,388,"Montgomery-Selma-Alexander City, AL",N,N,102,1088,1030,2021-2022
2,-86.472061,32.460800,3,A0900032,FIRST PRESBYTERIAN KINDERGARTEN,211 S CHESTNUT ST,PRATTVILLE,AL,36067,1,...,"Montgomery, AL",1,388,"Montgomery-Selma-Alexander City, AL",N,N,102,1088,1030,2021-2022
3,-86.466175,32.455425,4,A1592005,NEW HOPE ACADEMY,301 WATER ST,PRATTVILLE,AL,36067,1,...,"Montgomery, AL",1,388,"Montgomery-Selma-Alexander City, AL",N,N,102,1088,1030,2021-2022
4,-86.470441,32.461710,5,K9300030,FIRST BAPTIST PRESCHOOL AND KINDERGARTEN,138 S WASHINGTON ST,PRATTVILLE,AL,36067,1,...,"Montgomery, AL",1,388,"Montgomery-Selma-Alexander City, AL",N,N,102,1088,1030,2021-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22340,-110.810241,43.491060,22341,A0303404,MOUNTAIN ACADEMY OF TETON SCIENCE SCHOOLS,700 COYOTE CANYON RD,JACKSON,WY,83001,56,...,"Jackson, WY-ID",2,N,N,N,N,5600,56023,56017,2021-2022
22341,-110.799526,43.458650,22342,A0903887,JACKSON HOLE COMMUNITY SCHOOL,"1715 HIGH SCHOOL ROAD, #240",JACKSON,WY,83001,56,...,"Jackson, WY-ID",2,N,N,N,N,5600,56016,56017,2021-2022
22342,-110.844286,43.367002,22343,A1102418,RED TOP MEADOWS,7905 FALL CREEK RD,WILSON,WY,83014,56,...,"Jackson, WY-ID",2,N,N,N,N,5600,56022,56016,2021-2022
22343,-110.810901,43.441910,22344,A2000010,JACKSON HOLE CLASSICAL ACADEMY,2500 S PARK LOOP RD,JACKSON,WY,83001,56,...,"Jackson, WY-ID",2,N,N,N,N,5600,56023,56017,2021-2022


In [109]:
import pandas as pd 
df=pd.read_csv('superfund.csv')
df

,Region,State,Site Name,Site ID,EPA ID,Address,City,Zip,County,FF Ind,NAI,Native American Entity (NAI Status),Latitude,Longitude,NPL Status Date
0,1.0,CT,BARKHAMSTED-NEW HARTFORD LANDFILL,100255.0,CTD980732333,ROUTE 44,BARKHAMSTED,6063.0,LITCHFIELD,N,N,NaN,41.893947,-72.989337,1989-10-04 0:00:00
1,1.0,CT,BEACON HEIGHTS LANDFILL,100180.0,CTD072122062,BLACKBERRY HILL ROAD,BEACON FALLS,6403.0,NEW HAVEN,N,N,NaN,41.431950,-73.035281,1983-09-08 0:00:00
2,1.0,CT,DURHAM MEADOWS,100108.0,CTD001452093,MAIN ST,DURHAM,6422.0,MIDDLESEX,N,N,NaN,41.481110,-72.681388,1989-10-04 0:00:00
3,1.0,CT,GALLUP'S QUARRY,100201.0,CTD108960972,ROUTE 12,PLAINFIELD,6374.0,WINDHAM,N,N,NaN,41.665281,-71.924161,1989-10-04 0:00:00
4,1.0,CT,KELLOGG-DEERING WELL FIELD,100252.0,CTD980670814,NORWALK WATER DEPARTMENT,NORWALK,6856.0,FAIRFIELD,N,N,NaN,41.130550,-73.431950,1984-09-21 0:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,10.0,WA,"WESTERN PROCESSING CO., INC.",1000662.0,WAD009487513,7215 S 196TH ST,KENT,98031.0,KING,N,Y,Muckleshoot Indian Tribe,47.425000,-122.241700,1983-09-08 0:00:00
2168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(Current),NaN,NaN,NaN
2169,10.0,WA,WYCKOFF CO./EAGLE HARBOR,1000612.0,WAD009248295,5350 CREOSOTE PL NE,BAINBRIDGE ISLAND,98110.0,KITSAP,N,Y,Suquamish Indian Tribe of,47.621669,-122.516700,1987-07-22 0:00:00
2170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the Port Madison Reservation,NaN,NaN,NaN


In [110]:
import geopandas as gpd
from shapely.geometry import Point

df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)


In [111]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

In [112]:
gdf = gdf.to_crs(epsg=5070)
gdf['buffered'] = gdf.geometry.buffer(1500)
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
gdf_5070 = gdf.to_crs(epsg=5070)
gdf_5070.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [113]:
gdf.crs
gdf = gdf.to_crs(epsg=5070)
gdf['buffer_1500m'] = gdf.geometry.buffer(1000)

gdf = gdf.to_crs(epsg=5070)


gdf['original_point'] = gdf.geometry

gdf['buffer_1500m'] = gdf['original_point'].buffer(1500)
gdf = gdf.set_geometry('buffer_1500m')

### Using epsg=5070 for private school dataframe, which is df_4

In [115]:
geometry = [Point(xy) for xy in zip(df_4['Longitude'], df_4['Latitude'])]
gdf_privateschool = gpd.GeoDataFrame(df_4, geometry=geometry, crs="EPSG:4326")
gdf_privateschool = gdf_privateschool.to_crs(epsg=5070)
gdf_5070_school = gdf_privateschool.to_crs(epsg=5070)
gdf_5070_school.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [117]:
joined = gpd.sjoin(gdf_privateschool, gdf, how='left', predicate='intersects')
joined

,X,Y,OBJECTID,PPIN,NAME,STREET,CITY,STATE,ZIP,STFIP,...,Zip,County,FF Ind,NAI,Native American Entity (NAI Status),Latitude_right,Longitude_right,NPL Status Date,geometry_right,original_point
0,-86.541877,32.472090,1,2722,AUTAUGA ACADEMY,497 GOLSON RD,PRATTVILLE,AL,36067,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,-86.458475,32.469350,2,A0100060,CAMELLIA BAPTIST WEM,201 WOODVALE RD,PRATTVILLE,AL,36067,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,-86.472061,32.460800,3,A0900032,FIRST PRESBYTERIAN KINDERGARTEN,211 S CHESTNUT ST,PRATTVILLE,AL,36067,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,-86.466175,32.455425,4,A1592005,NEW HOPE ACADEMY,301 WATER ST,PRATTVILLE,AL,36067,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,-86.470441,32.461710,5,K9300030,FIRST BAPTIST PRESCHOOL AND KINDERGARTEN,138 S WASHINGTON ST,PRATTVILLE,AL,36067,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22340,-110.810241,43.491060,22341,A0303404,MOUNTAIN ACADEMY OF TETON SCIENCE SCHOOLS,700 COYOTE CANYON RD,JACKSON,WY,83001,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
22341,-110.799526,43.458650,22342,A0903887,JACKSON HOLE COMMUNITY SCHOOL,"1715 HIGH SCHOOL ROAD, #240",JACKSON,WY,83001,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
22342,-110.844286,43.367002,22343,A1102418,RED TOP MEADOWS,7905 FALL CREEK RD,WILSON,WY,83014,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
22343,-110.810901,43.441910,22344,A2000010,JACKSON HOLE CLASSICAL ACADEMY,2500 S PARK LOOP RD,JACKSON,WY,83001,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [118]:
joined['distance_to_site_m'] = joined.geometry.distance(joined['original_point'])
joined['distance_to_site_m']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
22340   NaN
22341   NaN
22342   NaN
22343   NaN
22344   NaN
Name: distance_to_site_m, Length: 22382, dtype: float64

In [119]:
closest = joined.sort_values('distance_to_site_m')
closest

,X,Y,OBJECTID,PPIN,NAME,STREET,CITY,STATE,ZIP,STFIP,...,County,FF Ind,NAI,Native American Entity (NAI Status),Latitude_right,Longitude_right,NPL Status Date,geometry_right,original_point,distance_to_site_m
13799,-73.733811,40.782128,13800,A0302252,LONG ISLAND HEBREW ACADEMY,122 CUTTERMILL RD,GREAT NECK,NY,11021,36,...,NASSAU,N,N,NaN,40.782361,-73.734139,1999-05-10 0:00:00,POINT (1846657.713 2191946.005),POINT (1846657.713 2191946.005),37.876403
5026,-80.209981,25.830920,5027,BB200435,MERRY POPPINS DAYCARE KINDERGARTEN INC,"790 NW 60TH STREET, 6001 NW 8TH AVE",MIAMI,FL,33127,12,...,MIAMI-DADE,N,N,NaN,25.831842,-80.210178,2012-03-15 0:00:00,POINT (1590968.422 441751.785),POINT (1590968.422 441751.785),103.117371
11751,-90.618641,38.604390,11752,K9303352,HOPE MONTESSORI ACADEMY WILDWOOD,16554 CLAYTON RD,WILDWOOD,MO,63011,29,...,ST. LOUIS,N,N,NaN,38.603331,-90.618331,1983-09-08 0:00:00,POINT (464027.008 1743592.219),POINT (464027.008 1743592.219),121.674897
4853,-80.208911,25.831850,4854,A1790127,B. WRIGHT LEADERSHIP ACADEMY,6100 NW 7TH AVE,MIAMI,FL,33127,12,...,MIAMI-DADE,N,N,NaN,25.831842,-80.210178,2012-03-15 0:00:00,POINT (1590968.422 441751.785),POINT (1590968.422 441751.785),128.254098
2908,-121.998351,37.381050,2909,A1770395,KINDERCARE LEARNING CENTER,1155 E ARQUES AVE,SUNNYVALE,CA,94085,6,...,SANTA CLARA,N,N,NaN,37.381600,-121.996800,1987-07-22 0:00:00,POINT (-2251862.417 1903456.632),POINT (-2251862.417 1903456.632),149.350895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22340,-110.810241,43.491060,22341,A0303404,MOUNTAIN ACADEMY OF TETON SCIENCE SCHOOLS,700 COYOTE CANYON RD,JACKSON,WY,83001,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
22341,-110.799526,43.458650,22342,A0903887,JACKSON HOLE COMMUNITY SCHOOL,"1715 HIGH SCHOOL ROAD, #240",JACKSON,WY,83001,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
22342,-110.844286,43.367002,22343,A1102418,RED TOP MEADOWS,7905 FALL CREEK RD,WILSON,WY,83014,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN
22343,-110.810901,43.441910,22344,A2000010,JACKSON HOLE CLASSICAL ACADEMY,2500 S PARK LOOP RD,JACKSON,WY,83001,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN


In [120]:
df = closest.dropna(subset=['distance_to_site_m'])
df

,X,Y,OBJECTID,PPIN,NAME,STREET,CITY,STATE,ZIP,STFIP,...,County,FF Ind,NAI,Native American Entity (NAI Status),Latitude_right,Longitude_right,NPL Status Date,geometry_right,original_point,distance_to_site_m
13799,-73.733811,40.782128,13800,A0302252,LONG ISLAND HEBREW ACADEMY,122 CUTTERMILL RD,GREAT NECK,NY,11021,36,...,NASSAU,N,N,NaN,40.782361,-73.734139,1999-05-10 0:00:00,POINT (1846657.713 2191946.005),POINT (1846657.713 2191946.005),37.876403
5026,-80.209981,25.830920,5027,BB200435,MERRY POPPINS DAYCARE KINDERGARTEN INC,"790 NW 60TH STREET, 6001 NW 8TH AVE",MIAMI,FL,33127,12,...,MIAMI-DADE,N,N,NaN,25.831842,-80.210178,2012-03-15 0:00:00,POINT (1590968.422 441751.785),POINT (1590968.422 441751.785),103.117371
11751,-90.618641,38.604390,11752,K9303352,HOPE MONTESSORI ACADEMY WILDWOOD,16554 CLAYTON RD,WILDWOOD,MO,63011,29,...,ST. LOUIS,N,N,NaN,38.603331,-90.618331,1983-09-08 0:00:00,POINT (464027.008 1743592.219),POINT (464027.008 1743592.219),121.674897
4853,-80.208911,25.831850,4854,A1790127,B. WRIGHT LEADERSHIP ACADEMY,6100 NW 7TH AVE,MIAMI,FL,33127,12,...,MIAMI-DADE,N,N,NaN,25.831842,-80.210178,2012-03-15 0:00:00,POINT (1590968.422 441751.785),POINT (1590968.422 441751.785),128.254098
2908,-121.998351,37.381050,2909,A1770395,KINDERCARE LEARNING CENTER,1155 E ARQUES AVE,SUNNYVALE,CA,94085,6,...,SANTA CLARA,N,N,NaN,37.381600,-121.996800,1987-07-22 0:00:00,POINT (-2251862.417 1903456.632),POINT (-2251862.417 1903456.632),149.350895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10648,-85.407951,41.796280,10649,2038189,TRINITY LUTHERAN SCHOOL,406 S LAKEVIEW AVE,STURGIS,MI,49091,26,...,ST. JOSEPH,N,N,NaN,41.798800,-85.425600,1984-09-21 0:00:00,POINT (870621.526 2136890.355),POINT (870621.526 2136890.355),1483.171026
4885,-80.224583,25.829253,4886,A2100813,AMC ACADEMY,5861 NW 17TH AVE,MIAMI,FL,33142,12,...,MIAMI-DADE,N,N,NaN,25.831842,-80.210178,2012-03-15 0:00:00,POINT (1590968.422 441751.785),POINT (1590968.422 441751.785),1485.573420
11057,-93.165683,44.962479,11058,A2102145,CHILDREN'S CENTER MONTESSORI INC.,1536 MINNEHAHA AVE W,SAINT PAUL,MN,55104,27,...,RAMSEY,N,N,NaN,44.972219,-93.152781,1983-09-08 0:00:00,POINT (224258.241 2445988.646),POINT (224258.241 2445988.646),1485.954667
22107,-87.895221,42.827623,22108,1512619,TRINITY LUTHERAN SCHOOL,7900 NICHOLSON RD,CALEDONIA,WI,53108,55,...,RACINE,N,N,NaN,42.835000,-87.880000,1987-07-22 0:00:00,POINT (659222.07 2232447.813),POINT (659222.07 2232447.813),1486.717166


In [121]:
df.to_csv('privateschools_near_npl')